In [7]:
import numpy as np
import tensorflow as tf
import itertools
import pandas as pd
# Own library
import test2
# Needed to clean memory
import gc

/home/rho/miniconda3/envs/env_DL/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/rho/miniconda3/envs/env_DL/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/rho/miniconda3/envs/env_DL/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/rho/miniconda3/envs/env_DL/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [1]:
# To modify scipts remotly
%load_ext autoreload
%autoreload 2

np.random.seed(94)
tf.random.set_seed(94)
(_, _), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_test = x_test / 255.0
x_test, y_test = tf.constant(x_test), tf.constant(y_test)

n = 1
x_test = x_test[:n]
y_test = y_test[:n]

path = 'data/model_cifar.h5'
model = tf.keras.models.load_model(path)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Parameter space
list_eta = np.arange(0, 1, 0.2)
list_epsilon = np.arange(0, 1, 0.2)
list_n_steps = np.arange(1, 5, 1)
list_methods = [test2.pgd_infinity, test2.pgd_ininity_random]

# Generate parameters grid
iter_parameters_grid = itertools.product(list_methods, list_eta, list_epsilon, list_n_steps)


# Names of methods
list_method_names = ['pgd_infinity', 'pgd_random']

# Compute different attack sets

nested_accuracies = []
for method, eta, eps, n_steps in iter_parameters_grid:
    x_adv = method(x_test, y_test, model, loss, eta=eta, eps=eps, n_steps=n_steps)
    # Re-scaling image (they are normalized)
    x_adv = tf.clip_by_value(x_adv, 0, 1)
    list_eval = model.evaluate(x=x_adv, y=y_test)
    # Clear memory
    gc.collect()
    tf.keras.backend.clear_session()
    del x_adv
      
    nested_accuracies.append(list_eval)

index = pd.MultiIndex.from_product([list_method_names, list_eta, list_epsilon],
                                   names=['method', 'tau', 'epsilon'])

# Graphs when step size is big
index = pd.MultiIndex.from_product([list_method_names, list_eta, list_epsilon, list_n_steps],
                                   names=['method', 'eta', 'epsilon', 'n_steps'])

df_results = pd.DataFrame(data=nested_accuracies,
                          columns=['loss', 'accuracy'],
                          index=index)

(df_results.unstack('method').loc[:, ('accuracy','pgd_infinity')]
           .unstack('epsilon')
           .drop(columns=0.0)
           .drop(0.0)
#  .plot(figsize=(16, 9))
)

path = './data/df_results.pickle'
import pickle
with open(path, 'wb') as f:
    pickle.dump(df_results, f)

In [3]:
import numpy as np
np.power([2], [2,4])

array([ 4, 16])

In [8]:

list_eta = np.arange(0, 1, 0.2)
list_epsilon = np.arange(0, 1, 0.2)
list_n_steps = np.arange(1, 5, 1)
list_methods = [test2.pgd_infinity, test2.pgd_ininity_random]

# Generate parameters grid
iter_parameters_grid = itertools.product(list_methods, list_eta, list_epsilon, list_n_steps)



In [10]:
list(iter_parameters_grid)

[(<function test2.pgd_infinity(x, y, model, loss_fn, eta=0.01, eps=0.1, n_steps=2)>,
  0.0,
  0.0,
  1),
 (<function test2.pgd_infinity(x, y, model, loss_fn, eta=0.01, eps=0.1, n_steps=2)>,
  0.0,
  0.0,
  2),
 (<function test2.pgd_infinity(x, y, model, loss_fn, eta=0.01, eps=0.1, n_steps=2)>,
  0.0,
  0.0,
  3),
 (<function test2.pgd_infinity(x, y, model, loss_fn, eta=0.01, eps=0.1, n_steps=2)>,
  0.0,
  0.0,
  4),
 (<function test2.pgd_infinity(x, y, model, loss_fn, eta=0.01, eps=0.1, n_steps=2)>,
  0.0,
  0.2,
  1),
 (<function test2.pgd_infinity(x, y, model, loss_fn, eta=0.01, eps=0.1, n_steps=2)>,
  0.0,
  0.2,
  2),
 (<function test2.pgd_infinity(x, y, model, loss_fn, eta=0.01, eps=0.1, n_steps=2)>,
  0.0,
  0.2,
  3),
 (<function test2.pgd_infinity(x, y, model, loss_fn, eta=0.01, eps=0.1, n_steps=2)>,
  0.0,
  0.2,
  4),
 (<function test2.pgd_infinity(x, y, model, loss_fn, eta=0.01, eps=0.1, n_steps=2)>,
  0.0,
  0.4,
  1),
 (<function test2.pgd_infinity(x, y, model, loss_fn, et